In [1]:
from Process_Function import *
from Process_Pipeline import *
from Process_F import *
from split_speaker import *
from model_GMM import *
from docx import Document

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
[NeMo W 2022-08-25 10:36:50 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


Input/Speaker_wav/Minh Thoa.wav


In [2]:
from docx import Document
def Create_Word(Docx_Name,final_text):
    os.makedirs('./ConvertWord/',exist_ok=True)
    doc = Document()
    doc.add_heading('Conversation',level=0)
    for sen in final_text:
        doc.add_paragraph(sen.strip())
    doc.save(Docx_Name)

# Transcribe Audio with Split Function
Purpose: To trim long audio, which's duration is longer than 1 minutes. With that, the processing will faster. For audio longer than 1 minutes, you need to send it to Google Cloud, it may take long time for very long audio.

Split Function: In this implementation, I have two function
<ul>
<li>Split Audio by silences with <a href="https://github.com/jiaaro/pydub">pydub.AudioSegment</a>
</li>
<li>Split Audio by speakers with speaker diarization from <a href="https://github.com/pyannote/pyannote-audio">Pyannote</a>
</li>
<li>Split Audio by speakers with speaker diarization from  
<a href="https://github.com/pyannote/pyannote-audio">Agglomerative Clustering Algorithm</a>
</li>
</ul>

# Example 1: 
https://vov.vn/the-gioi/australia-lap-dat-mat-than-theo-doi-nam-cuc-post965108.vov

## Split speakers with Agglomerative Clustering Algorithm
This method require K, the number of speaker. I set default equal 2. So that, despite this example just have 1 voice, this one will be splited to 2 speaker.

In [3]:
Audio_addr = r'Input\Australia lap mat than  theo doi Nam Cuc_23-08-2022-10-37-58.wav'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)

Func_Pipeline(aPipeline,Split_speaker_GMM,set_channel=True)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels = True)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex1_Split_speaker_GMM.docx'
doc.save(Docx_)

	 AudioNode  
Training GMM..
Training Done
[{'Speaker': 'Speaker 0', 'Start': 750.0, 'End': 7970.0}, {'Speaker': 'Speaker 1', 'Start': 7970.0, 'End': 11620.0}, {'Speaker': 'Speaker 0', 'Start': 11620.0, 'End': 67710.0}, {'Speaker': 'Speaker 1', 'Start': 67710.0, 'End': 79930.0}, {'Speaker': 'Speaker 0', 'Start': 79930.0, 'End': 84040.0}]
Audio_process/0.wav
Speaker 0 :
hôm nay cơ quan Nam Cực thuộc bộ biến đổi khí hậu năng lượng môi trường và nước của Australia và thông báo cho biết trong

Speaker 1 :
cơ quan này sẽ lắp đặt hệ thống mắt thần có tên

Speaker 0 :
Tennis tại Nam Cực để theo dõi sự thay đổi của khí hậu tại khu vực này trong đó có mức độ tuyết rơi tốc độ gió và cả nhiệt độ cùng với các thông tin về thời tiết các mắt thần này cũng hỗ trợ việc quản lý và sử dụng hạ tầng cơ sở tại thực địa để xác định Xem các hạ tầng này có cần phải báo chị hay không là một hệ thống gồm 2 camera trong đó nha chính có chức năng siêu zoom 25x với góc quay 360 độ và một camera khác nhỏ hơn có thể

## Split speakers with Pyannote
This method will choose the number of speakers automatedly. But after run many sample, the previous one will split better than this one, if it have right K.

In [4]:
Audio_addr = r'Input\Australia lap mat than  theo doi Nam Cuc_23-08-2022-10-37-58.wav'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)

Func_Pipeline(aPipeline,Split_speaker,set_channel=False)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels = True)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex1_Split_speaker_Pyannote.docx'
doc.save(Docx_)

	 AudioNode  
Audio_process/0.wav
SPEAKER_00 :
hôm nay cơ quan Nam Cực thuộc bộ biến đổi khí hậu năng lượng môi trường và nước của Australia và thông báo cho biết trong những năm tới cơ quan này sẽ lắp đặt ba hệ thống mắt thần có tên gọi là em ít tại Nam Cực để theo dõi sự thay đổi của khí hậu tại khu vực này trong đó có mức độ tuyết rơi tốc độ gió và cả nhiệt độ

SPEAKER_00 :
cùng với các thông tin về thời tiết các nốt lần này cũng hỗ trợ việc quản lý và sử dụng hạ tầng cơ sở tại thực hiện để xác định Xem các hạ tầng này có cần phải báo chị hay không

SPEAKER_00 :
mắt thần anh em asp là một hệ thống gồm 2 camera trong đó camera kính có chức năng siêu zoom 25x với góc quay 360 độ và một camera khác nhỏ hơn có thể chuyển ra ngoài và đi vào trong bộ thiết bị

SPEAKER_00 :
các camera này được đặt trên một khung nhôm và thép hình vuông nặng 650 kg cùng với các giá đỡ và bằng điều khiển năng lượng mặt trời và một anken bạn tinh

SPEAKER_00 :
vẫn bảo sao không như này là 1 một kim loại cao 4

## Split silences with silences time no longer than 500 ms

In [5]:
Audio_addr = r'Input\Australia lap mat than  theo doi Nam Cuc_23-08-2022-10-37-58.wav'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)

Func_Pipeline(aPipeline,Split_silence,set_channel=False)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels = True)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex1_Split_silences_500ms.docx'
doc.save(Docx_)

	 AudioNode  
Audio_process/0.wav
hôm nay cơ quan Nam Cực thuộc bộ biến đổi khí hậu năng lượng môi trường và nước của Australia và thông báo cho biết

trong những năm tới cơ quan này sẽ lắp đặt ba hệ thống mắt thần có tên gọi là em ít tại Nam Cực

để theo dõi sự thay đổi của khí hậu tại khu vực này

trong đó có mức độ tuyết rơi tốc độ gió và cả nhiệt độ

cùng với các thông tin về thời tiết các nốt lần này cũng hỗ trợ việc quản lý và sử dụng hạ tầng cơ sở tại thực hiện

để xác định Xem các hạ tầng này có cần phải báo chị hay không

mắt thần Artemis là một hệ thống gồm 2 camera

trong đó camera kính có chức năng siêu zoom 25x với góc quay 360 độ

và một camera khác nhỏ hơn có thể di chuyển ra ngoài và đi vào trong bộ thiết bị

các camera này được đặt trên một khung nhôm và thép hình vuông nặng 650 kg cùng với các giá đỡ và bằng điều khiển năng lượng mặt trời và một anken bạn tinh

vẫn bảo sao không hôm này là 1 một kim loại cao 4 m và là nơi gắn 2 camera

hệ thống mắt thần Artemis đầu ti

# Example 2:
https://www.qdnd.vn/giao-duc-khoa-hoc/tin-tuc/giao-vien-chua-dat-chuan-o-cap-tieu-hoc-chiem-ty-le-cao-nhat-702958

## Split speakers with Agglomerative Clustering Algorithm

In [6]:
Audio_addr = r'Input\giao-vien-chua-dat-chuan-o-cap-tieu-hoc-chiem-ty-le-cao-nhat.m4a'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)
aPipeline.To_Channel(1)
Func_Pipeline(aPipeline,Split_speaker_GMM,set_channel=False)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels=False)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex2_Split_speaker_GMM.docx'
doc.save(Docx_)

	 AudioNode  
Training GMM..
Training Done
[{'Speaker': 'Speaker 1', 'Start': 260.0, 'End': 4830.0}, {'Speaker': 'Speaker 0', 'Start': 4830.0, 'End': 11500.0}, {'Speaker': 'Speaker 1', 'Start': 11500.0, 'End': 28960.0}, {'Speaker': 'Speaker 0', 'Start': 28960.0, 'End': 42710.0}]
Audio_process/0.wav
Speaker 1 :
anh em mình chưa đạt chuẩn ở cấp tiểu học chiếm tỷ lệ cao nhất

Speaker 0 :
Theo thống kê của Bộ Giáo dục và Đào tạo huyện có 25,2% giáo viên tiểu học cả nước chưa đạt chuẩn trình độ đào

Speaker 1 :
cụ thể Chị là giáo viên đạt chuẩn về trình độ đào tạo theo luật giáo dục 2019 ở cấp Mầm non là 90,7% tiểu học là 74,8% trung học cơ sở là 86,1% trung học phổ thông là 99,9%

Speaker 0 :
như vậy cách tiểu học có tỉ lệ giáo viên chưa đạt chuẩn cao nhất với 25,2% tiếp đó là cách trung học cơ sở với 13,9% cấp Mầm non là 8,3% cấp trung học phổ thông có tỉ lệ giáo viên chưa đạt chuẩn chị 0,1%



## Split speakers with Pyannote

In [7]:
Audio_addr = r'Input\giao-vien-chua-dat-chuan-o-cap-tieu-hoc-chiem-ty-le-cao-nhat.m4a'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)
aPipeline.To_Channel(1)
Func_Pipeline(aPipeline,Split_speaker,set_channel=False)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels=False)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex2_Split_speaker_Pyannote.docx'
doc.save(Docx_)

	 AudioNode  
Audio_process/0.wav
SPEAKER_00 :
anh em mình chưa đạt chuẩn ở cấp tiểu học chiếm tỷ lệ cao nhất

SPEAKER_00 :
Theo thống kê của Bộ Giáo dục và Đào tạo huyện có 25,2% giáo viên tiểu học cả nước chưa đạt chuẩn trình độ đào tạo

SPEAKER_00 :
cụ thể chị tôi là giáo viên đạt chuẩn về trình độ đào tạo theo luật giáo dục 2019 ở cấp Mầm non là 90,7% tiểu học là 74,8% trung học cơ sở là 66,1% trung học là 99,9%

SPEAKER_00 :
như vậy cách tiểu học có tỉ lệ giáo viên chưa đạt chuẩn cao nhất với 25,2% tiếp đó là cách trung học cơ sở với 13,9% cấp Mầm non là 8,3% cấp trung học phổ thông có tỉ lệ giáo viên chưa đạt chuẩn chị 0,1%



## Split silences with silences time no longer than 500 ms

In [8]:
Audio_addr = r'Input\giao-vien-chua-dat-chuan-o-cap-tieu-hoc-chiem-ty-le-cao-nhat.m4a'
Audio_addr = to_wav(Audio_addr)
Audio_addr = Link_Audio(Audio_addr)

aPipeline = Init_Pipeline(Audio_addr)
aPipeline.To_Channel(1)
Func_Pipeline(aPipeline,Split_silence,set_channel=False)

info,HashList = aPipeline.prepare_to_S2t(-1,set_channels = False)

S2t(HashList)
StringList = S2t_get_str(HashList)
S2t_filter(StringList)
final_text = S2t_print(StringList)
doc = Document()
for sen in final_text:
    doc.add_paragraph(sen.strip())
Docx_ = 'Ex2_Split_silences_500ms.docx'
doc.save(Docx_)

	 AudioNode  
Audio_process/0.wav
anh em mình chưa đạt chuẩn ở cấp tiểu học chiếm tỷ lệ cao nhất theo thống kê của Bộ Giáo dục và Đào tạo huyện có 25,2% giáo viên tiểu học

cả nước chưa đạt chuẩn trình độ đào tạo,  cụ thể Chị là giáo viên đạt chuẩn về trình độ đào tạo theo luật giáo dục 2019 ở cấp Mầm non là 90,7% tiểu học là 74,8% trung học cơ sở là 66,1% trung học phí là 99,9%

như vậy cấp tiểu học có tỉ lệ giáo viên chưa đạt chuẩn cao nhất với 25,2% tiếp đó là cấp trung học cơ sở với 13,9% cấp Mầm non là 8,3% cấp trung học phổ thông có tỉ lệ giáo viên chưa đạt chuẩn

